# Projet

## Imports

In [1]:
import copy
import wave
from pathlib import Path
from os import path, makedirs
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Conv1D, AvgPool1D, MaxPool1D, ZeroPadding1D, BatchNormalization, Flatten, Dense, Activation
from keras.utils.np_utils import to_categorical
from mutagen.wave import WAVE
from mutagen.mp3 import MP3
from pydub import AudioSegment
from random import choice

C:\Users\Vinh\AppData\Local\Programs\Python\Python310\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


## Récupération des données

## Traitement des données

In [ ]:
# A modifier
def split_sounds(sound_data):
    curr_path = sound_data["curr_path"]
    absolute_path = path.dirname(__file__)
    dest_dirname = sound_data['dest_dirname']
    dest_filename, extension = path.splitext(sound_data['dest_filename'])
    audio_length = int(WAVE(curr_path).info.length)
    audio = AudioSegment.from_wav(curr_path)

    relative_path = f"{DEST_DIR}/testing_list.txt"
    absolute_path = path.dirname(__file__)
    testing_file_path = path.join(absolute_path, relative_path)

    with open(testing_file_path,"a+") as file:
        for i in range(audio_length):
            t1 = i * 1000
            t2 = t1 + 1000
            cut_audio = audio[t1:t2]
            relative_path = f"{DEST_DIR}/{dest_dirname}/{dest_filename}_{i}{extension}"
            dest_path = path.join(absolute_path, relative_path)
            makedirs(path.dirname(dest_path), exist_ok=True)
            cut_audio.export(dest_path, format="wav")
            if choice([True, False, False]):
                file.write(f'{dest_dirname}/{dest_filename}_{i}{extension}\n')

## Création des jeux de données

In [ ]:
dataset_dir = Path('recordings')

CLASSES = ["Parus major", "Turdus merula", "Fringilla-coelebs"]

with (dataset_dir/ 'testing_list.txt').open() as f:
    testing_list = f.read().splitlines()

x_train = []
y_train = []
x_test = []
y_test = []

for recording in dataset_dir.glob('**/*.wav'):
    if not recording.parent.name in CLASSES:
        continue
    label = CLASSES.index(recording.parent.name)

    with wave.open(str(recording)) as f :
        data = np.frombuffer(f.readframes(f.getnframes()), dtype=np.int16).copy()

    data = data.astype(np.float32)
    data.resize((16000, 1))

    if str(recording.relative_to(dataset_dir)) in testing_list:
        x_test.append(data)
        y_test.append(label)
    else:
        x_train.append(data)
        y_train.append(label)

x_train = np.array(x_train)
y_train = to_categorical(np.array(y_train))
x_test = np.array(x_test)
y_test = to_categorical(np.array(y_test))

## Normalize data

In [ ]:
x_mean = x_train.mean()
x_std = x_train.std()

x_train -= x_mean
x_test -= x_mean
x_train /= x_std
x_test /= x_std

## Exporter les données

In [ ]:
np.savetxt('x_test.csv', x_test.reshape(x_test.shape[0], -1), delimiter=',', fmt='%s')
np.savetxt('y_test.csv', y_test, delimiter=',', fmt='%s')

## Build model M5

In [ ]:
# Modifier

model = Sequential()
model.add(Input(shape=(16000, 1)))
model.add(Conv1D(filters=128, kernel_size=80, strides=4, activation='relu'))
model.add(MaxPool1D(pool_size=4, strides=1, padding='valid'))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4, strides=1, padding='valid'))
model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4, strides=1, padding='valid'))
model.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4, strides=1, padding='valid'))
model.add(AvgPool1D())
model.add(Flatten())
model.add(Dense(units=10))
model.add(Activation('softmax'))  # SoftMax activation needs to be separate from Dense to remove it later on
# EXPLORE Learning Rate
opt = tf.keras.optimizers.Adam(learning_rate=10e-3)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

## Train model

In [ ]:
model.fit(x_train, y_train, epochs=5, batch_size=10, validation_data=(x_test, y_test))

## Evaluate model on test dataset

In [ ]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

## Evaluate model on small dataset

In [ ]:
model.evaluate(x_test, y_test, verbose=2)
pred_test_250 = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test_250.argmax(axis=1)))

## Save trained model

In [ ]:
model.save('lab_gsc.h5')

## Remove SoftMax layer

In [ ]:
model = tf.keras.Model(model.input, model.layers[-2].output, name=model.name)

## Install MicroAI for C inference code generation (kerascnn2c module)

In [ ]:
!pip install https://bitbucket.org/edge-team-leat/microai_public/get/6adfbcb347d3.zip#subdirectory=third_party/kerascnn2c_fixed
import kerascnn2c

## Generate C code for the trained model with 16-bit fixed-point representation

In [ ]:
res = kerascnn2c.Converter(output_path=Path('gsc_output_fixed'),
                           fixed_point=9, # Number of bits for the fractional part, Q7.9 format
                           number_type='int16_t', # Data type for weights/activations (16 bits quantization)
                           long_number_type='int32_t', # Data type for intermediate results
                           number_min=-(2**15), # Minimum value for 16-bit signed integers
                           number_max=(2**15)-1 # Maximum value for 16-bit signed integers
                          ).convert_model(copy.deepcopy(model))
with open('gsc_model_fixed.h', 'w') as f:
    f.write(res)

## Compile the 16-bit fixed-point C code for x86 and evaluate on small dataset

In [ ]:
!g++ -Wall -Wextra -pedantic -Ofast -o gsc_fixed -Igsc_output_fixed/ gsc_output_fixed/model.c main.cpp
!./gsc_fixed x_test_gsc_250.csv y_test_gsc_250.csv